### Problem 1

In [2]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import yfinance as yf

In [3]:
# ---------------------------------------------------------
# Step 1: Fetch Nifty50 data from NSE
# ---------------------------------------------------------
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Referer": "https://www.nseindia.com/"
})
session.get("https://www.nseindia.com")

nifty50_url = "https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%2050"
response = session.get(nifty50_url)
response.raise_for_status()
data = response.json()

symbols = [item['symbol'] for item in data['data']]

# If "NIFTY 50" is in symbols, remove it because it's not a stock symbol
# and Yahoo won't recognize "NIFTY 50.NS"
if "NIFTY 50" in symbols:
    symbols.remove("NIFTY 50")

nse_details = {}
for item in data['data']:
    sym = item['symbol']
    nse_details[sym] = {
        "LTP": item.get('lastPrice'),
        "Volume": item.get('totalTradedVolume'),
        "%Change": item.get('pChange'),
        "52W_High": item.get('yearHigh'),
        "52W_Low": item.get('yearLow'),
        "Day_High": item.get('dayHigh'),
        "Day_Low": item.get('dayLow')
    }

# ---------------------------------------------------------
# Step 2: Fetch PE, EPS, Market Cap from yfinance's stock.info
# ---------------------------------------------------------
info_data = {}
for sym in symbols:
    yahoo_symbol = sym + ".NS"
    try:
        stock = yf.Ticker(yahoo_symbol)
        info = stock.info
    except:
        info = {}
    pe = info.get('trailingPE', np.nan)
    eps = info.get('trailingEps', np.nan)
    market_cap = info.get('marketCap', np.nan)

    info_data[sym] = {
        "PE": pe,
        "EPS": eps,
        "Market_Cap": market_cap
    }


# ---------------------------------------------------------
# Step 3: Compute Returns from Yahoo Finance (6M, 1Y, 5Y)
# ---------------------------------------------------------
def get_returns(symbol):
    yahoo_symbol = symbol + ".NS"
    end = datetime.now()
    start_5y = end - timedelta(days=5 * 365)
    try:
        df_yf = yf.download(yahoo_symbol, start=start_5y, end=end, progress=False)
    except:
        return None, None, None

    if df_yf.empty:
        return None, None, None

    current_price = df_yf['Adj Close'].iloc[-1].item() if hasattr(df_yf['Adj Close'].iloc[-1], 'item') else \
    df_yf['Adj Close'].iloc[-1]

    # 6-month return
    start_6m = end - timedelta(days=182)
    df_6m = df_yf[df_yf.index >= start_6m]
    ret_6m = None
    if not df_6m.empty:
        old_6m = df_6m['Adj Close'].iloc[0].item() if hasattr(df_6m['Adj Close'].iloc[0], 'item') else \
        df_6m['Adj Close'].iloc[0]
        ret_6m = ((current_price - old_6m) / old_6m) * 100

    # 1-year return
    start_1y = end - timedelta(days=365)
    df_1y = df_yf[df_yf.index >= start_1y]
    ret_1y = None
    if not df_1y.empty:
        old_1y = df_1y['Adj Close'].iloc[0].item() if hasattr(df_1y['Adj Close'].iloc[0], 'item') else \
        df_1y['Adj Close'].iloc[0]
        ret_1y = ((current_price - old_1y) / old_1y) * 100

    # 5-year return
    old_5y = df_yf['Adj Close'].iloc[0].item() if hasattr(df_yf['Adj Close'].iloc[0], 'item') else \
    df_yf['Adj Close'].iloc[0]
    ret_5y = ((current_price - old_5y) / old_5y) * 100

    return ret_6m, ret_1y, ret_5y


returns_data = {}
for sym in symbols:
    returns_data[sym] = get_returns(sym)

# ---------------------------------------------------------
# Step 4: Combine all data into a DataFrame
# ---------------------------------------------------------
all_rows = []
for sym in symbols:
    rets = returns_data[sym]

    row = {
        "Symbol": sym,
        "LTP": nse_details[sym].get("LTP"),
        "Volume": nse_details[sym].get("Volume"),
        "%Change": nse_details[sym].get("%Change"),
        "PE": info_data[sym].get("PE"),
        "EPS": info_data[sym].get("EPS"),
        "Market_Cap": info_data[sym].get("Market_Cap"),
        "52W_High": nse_details[sym].get("52W_High"),
        "52W_Low": nse_details[sym].get("52W_Low"),
        "Upper_Circuit": nse_details[sym].get("Day_High"),
        "Lower_Circuit": nse_details[sym].get("Day_Low"),
        "6M_Return%": rets[0] if rets else np.nan,
        "1Y_Return%": rets[1] if rets else np.nan,
        "5Y_Return%": rets[2] if rets else np.nan
    }

    all_rows.append(row)

df = pd.DataFrame(all_rows)
print(df.head())

       Symbol       LTP    Volume  %Change         PE     EPS      Market_Cap  \
0  BHARTIARTL   1682.00  12900707     4.44  81.283226   20.69  10068587184128   
1         ITC    471.00  31753858     2.26  28.658537   16.40   5880076042240   
2   KOTAKBANK   1804.90   3893531     2.05  16.474909  109.60   3589921046528   
3  HINDUNILVR   2391.25   2260902     1.97  54.643350   43.74   5615755198464   
4  ULTRACEMCO  12082.35    408880     1.90  53.190865  227.18   3485226500096   

   52W_High  52W_Low  Upper_Circuit  Lower_Circuit  6M_Return%  1Y_Return%  \
0    1779.0   960.00        1685.00        1606.80   18.396895   70.777290   
1     528.5   399.35         474.40         451.65    9.620991    7.464088   
2    1942.0  1543.85        1809.00        1748.05    5.131385   -2.128963   
3    3035.0  2172.05        2394.55        2333.45   -3.137340   -4.093450   
4   12138.0  9250.00       12118.50       11730.00    9.321947   21.917101   

   5Y_Return%  
0  296.572930  
1  140.53833

### Problem 2